In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
import re
from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

### 도구 호출 (Tool Calling)
> 웹 검색 기능을 AI모델에 통합해주는 웹 검색 API

- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음

랭체인 내장 도구 : Tavily

In [31]:
# poetry add langchain-tavily
# 최신 langchain 0.3.25 최신버전 기준  
# langchain-tavily 포함된 클래스를 사용

from langchain_tavily.tavily_search import TavilySearch

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
web_search = TavilySearch(max_results=3)

# 웹 검색 실행
search_results = web_search.invoke(query)
print(type(search_results))
#pprint(search_results)

# 검색 결과 출력
for result in search_results['results']:
    print(type(result))
    pprint(result)  
    print("-" * 100)  

<class 'dict'>
<class 'dict'>
{'content': 'Published Time: 2022-01-27T11:22:15+09:00 스테이크와 어울리는 최고의 와인: 무엇을 '
            '고를 것인가? 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? Tags: 마시자매거진, 소고기와인, '
            '스테이크와인, 와인추천, 와인페어링 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 2018년 디캔터 월드 '
            '와인 어워드(Decanter World Wine Awards)에서 아르헨티나 지역 의장이었던 남미 와인 전문가이자 '
            '저널리스트인 파트리시오 타피아(Patricio Tapia)는 ‘나는 오크가 적고 신선한 과일과 더 좋은 산도를 가진 '
            '‘뉴 웨이브(new wave)’ 말벡을 선택하는 경향이 있다.’라고 말한다. ‘와인과 소고기를 페어링하는 가장 쉬운 '
            '방법은 소고기와 와인의 풍미 강도를 일치시키는 방법에 대해 생각하는 것이다.’라고 Hawksmoor 스테이크하우스 '
            '레스토랑의 와인 디렉터인 마크 퀵(Mark Quick)이 와인과 쇠고기의 페어링에 관한 심층 기사에서 언급했다. '
            'Tags: 마시자매거진 소고기와인 스테이크와인 와인추천 와인페어링 Bora Kim Previous Article '
            'Next Article Your email address will not be published. 대표자 : 방문송 '
            '사업자등록번호 : 325-87-00031 발행인 / 편집인 : 방문송',
 'raw_content': None,
 'score': 0.8668671,
 'title': '스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? - 마시자 매거진',
 'url': 'https://mashija.com/%E

In [32]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_tavily.tavily_search.TavilySearch'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. It not '
 'only retrieves URLs and snippets, but offers advanced search depths, domain '
 'management, time range filters, and image search, this tool delivers '
 'real-time, accurate, and citation-backed results.Input should be a search '
 'query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for [TavilySearch]',
 'properties': {'exclude_domains': {'anyOf': [{'items': {'type': 'string'},
                                               'type': 'array'},
        

### 도구 호출
- bind_tools로 LLM에 직접 바인딩

In [33]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(model="gpt-4o-mini")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=[web_search])
print(type(llm_with_tools))

<class 'langchain_core.runnables.base.RunnableBinding'>


In [34]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)
print(type(ai_msg))

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

<class 'langchain_core.messages.ai.AIMessage'>
AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 770, 'total_tokens': 781, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bnf5EkgXlB546JAknQQJtc1iERd8K', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--affc5e5c-0197-42c7-b88f-30228537bf41-0', usage_metadata={'input_tokens': 770, 'output_tokens': 11, 'total_tokens': 781, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
----------------------------------------------------------------------------------------------------
'안녕하세요! 어떻게 도와드릴까요?'
---------

In [35]:
from pprint import pprint

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# AIMessage의 속성 확인
#pprint(dir(ai_msg))

In [36]:
pprint(ai_msg)
print("#" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("*" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nEZkL99ORiwwFpQ2kdqCbKAi', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 779, 'total_tokens': 804, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bnf5F8YCMTgAsK73lHBoMwMh5hlQh', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3aafe45b-8486-430a-974f-18e731b69d12-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_nEZkL99ORiwwFpQ2kdqCbKAi', 'type': 'tool_call'}], usage_metadata={'input_tokens': 779, 'output_tokens': 25, 'total_tokens': 804,

### 도구 실행하기

#### 1. 직접 도구 호출 처리한 결과로 ToolMessage 생성
- 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
- 'args' 속성을 사용하여 TavilySearch 객체의 invoke() 호출하여 결과를 얻는다.

In [37]:
tool_call = ai_msg.tool_calls[0]

tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)

pprint(tool_output)

tavily_search 호출 결과:
----------------------------------------------------------------------------------------------------
{'answer': None,
 'follow_up_questions': None,
 'images': [],
 'query': '스테이크와 어울리는 와인 추천',
 'response_time': 1.45,
 'results': [{'content': 'Published Time: 2022-01-27T11:22:15+09:00 스테이크와 어울리는 '
                         '최고의 와인: 무엇을 고를 것인가? 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? '
                         'Tags: 마시자매거진, 소고기와인, 스테이크와인, 와인추천, 와인페어링 스테이크와 어울리는 '
                         '최고의 와인: 무엇을 고를 것인가? 2018년 디캔터 월드 와인 어워드(Decanter '
                         'World Wine Awards)에서 아르헨티나 지역 의장이었던 남미 와인 전문가이자 '
                         '저널리스트인 파트리시오 타피아(Patricio Tapia)는 ‘나는 오크가 적고 신선한 과일과 '
                         '더 좋은 산도를 가진 ‘뉴 웨이브(new wave)’ 말벡을 선택하는 경향이 있다.’라고 '
                         '말한다. ‘와인과 소고기를 페어링하는 가장 쉬운 방법은 소고기와 와인의 풍미 강도를 일치시키는 '
                         '방법에 대해 생각하는 것이다.’라고 Hawksmoor 스테이크하우스 레스토랑의 와인 디렉터인 '
                         '마크 퀵(Mark Quick)이 와인과 쇠

#### (추천) 방법 2: 도구 직접 호출하여 바로 ToolMessage 객체 생성
- tool_call 변수는 AIMessage 객체 포함된 tool 을 호출한 결과 tool_calls의 첫번째 dict 객체
- TavilySearch 객체를 invoke(tool_call) 하면 결과객체의 타입이 ToolMessage 이다.
- 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
- 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

In [38]:
# tool_call  {'name': 'tavily_search_results_json', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_LrHyxTadqHDjW7J6LOWEaoSi', 'type': 'tool_call'}
tool_message = web_search.invoke(tool_call)
print(type(tool_message))

# 특정 속성들만 확인
print("\n=== 주요 속성들 ===")
attributes = ['content', 'tool_call_id', 'name', 'type', 'additional_kwargs']
for attr in attributes:
    if hasattr(tool_message, attr):
        print(f"{attr}: {getattr(tool_message, attr)}")

print("-" * 100)
pprint(tool_message.content)

<class 'langchain_core.messages.tool.ToolMessage'>

=== 주요 속성들 ===
content: {"query": "스테이크와 어울리는 와인 추천", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/", "title": "스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? - 마시자 매거진", "content": "Published Time: 2022-01-27T11:22:15+09:00 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? Tags: 마시자매거진, 소고기와인, 스테이크와인, 와인추천, 와인페어링 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 2018년 디캔터 월드 와인 어워드(Decanter World Wine Awards)에서 아르헨티나 지역 의장이었던 남미 와인 전문가이자 저널리스트인 파트리시오 타피아(Patricio Tapia)는 ‘나는 오크가 적고 신선한 과일과 더 좋은 산도를 가진 ‘뉴 웨이브(new wave)’ 말벡을 선택하는 경향이 있다.’라고 말한다. ‘와인과 소고기를 페어링하는 가장 쉬운 방법은 소고기와 와인의 풍미 강도를 일치시키는 방법에 대해 생각하는 것이다.’라고 Hawksmoor 스테이크하우스 레스토랑의 와인 디렉터인 마크 퀵(Mark Quick)이 와인과 쇠고기의 페어링에 관한 심층 기사에서 언급했다. Tags: 마시자매거진 소고기와인 스

#### 도구 호출이 여러개인 경우 - batch 사용

In [39]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

[ToolMessage(content='{"query": "스테이크와 어울리는 와인 추천", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/", "title": "스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? - 마시자 매거진", "content": "Published Time: 2022-01-27T11:22:15+09:00 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? Tags: 마시자매거진, 소고기와인, 스테이크와인, 와인추천, 와인페어링 스테이크와 어울리는 최고의 와인: 무엇을 고를 것인가? 2018년 디캔터 월드 와인 어워드(Decanter World Wine Awards)에서 아르헨티나 지역 의장이었던 남미 와인 전문가이자 저널리스트인 파트리시오 타피아(Patricio Tapia)는 ‘나는 오크가 적고 신선한 과일과 더 좋은 산도를 가진 ‘뉴 웨이브(new wave)’ 말벡을 선택하는 경향이 있다.’라고 말한다. ‘와인과 소고기를 페어링하는 가장 쉬운 방법은 소고기와 와인의 풍미 강도를 일치시키는 방법에 대해 생각하는 것이다.’라고 Hawksmoor 스테이크하우스 레스토랑의 와인 디렉터인 마크 퀵(Mark Quick)이 와인과 쇠고기의 페어링에 관한 심층 기사에서 언급했다. Tags: 마시자매거진 소고기와인 스테이크와인 와인추천 와인페어링 Bora Kim Previous Article Next Articl

### ToolMessage를 LLM에 전달하여 답변 생성

In [40]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_community.tools import TavilySearchResults

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")
print(type(llm))
# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)

tavily_tool = TavilySearchResults(max_results=2)
print(type(tavily_tool))

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_tool])
print(type(llm_with_tools))
#<class 'langchain_core.runnables.base.RunnableBinding'>

# LLM 체인 생성
llm_chain = prompt | llm_with_tools
print(type(llm_chain))

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
<class 'langchain_core.runnables.base.RunnableBinding'>
<class 'langchain_core.runnables.base.RunnableSequence'>


#### Chain decorator

In [41]:
# 도구 실행 체인 정의
# @chain 은 일반 함수를 Runnable 객체로 만들어주는 데코레이터 
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print(type(ai_msg))
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = tavily_tool.batch(ai_msg.tool_calls, config=config)
    print(type(tool_msgs))
    print("tool_msgs: \n", tool_msgs)
    
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
query = "스테이크와 어울리는 와인을 추천해주세요."
response = web_search_chain.invoke(query)

# 응답 출력 
pprint(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_12yxPUQ9CEjpQLgZfPLksNUM', 'function': {'arguments': '{"query": "스테이크 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_c8v7CsuqfsTHmBG4EZIHabeB', 'function': {'arguments': '{"query": "스테이크 음식과 어울리는 와인"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 109, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-Bnf5NU41OMx0W1RV5iS08faB5QEuD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--0b2297e5-4065-49d1-a62c-22e1ad1d0606-0' tool_ca

#### Tool decorator

In [42]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [43]:

from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)
print(type(wiki_search))

<class 'langchain_core.tools.structured.StructuredTool'>


In [44]:
# 위키 검색 실행
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(type(result))
    print(result)  
    print("-" * 100)  

<class 'langchain_core.documents.base.Document'>
page_content='오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.
터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 만들거나 초르바(수프)에 넣어 먹는다.


== 같이 보기 ==
파스티나
셰흐리예


== 각주 ==' metadata={'title': '오르조', 'summary': '오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.\n터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 

In [45]:

# LLM에 도구를 바인딩 (2개의 도구 바인딩)
print(type(tavily_search_func)) # @tool 선언
print(type(wiki_search)) # as_tool() 함수사용

llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

<class 'langchain_core.tools.structured.StructuredTool'>
<class 'langchain_core.tools.structured.StructuredTool'>
AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uFAGaO57yaelMOfcfIPwxhhi', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_c5l4Bej0YNzwS9pOASy6Iu6r', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 176, 'total_tokens': 234, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bnf5zwSdEQrbRfGCBlogEHpDjitdJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bab4d73

### LCEL을 통해 위키피디아 문서를 검색하고 내용을 요약

In [46]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarized_text)

('오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래한 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 '
 '박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. 이탈리아 요리는 기원전 4세기부터 발전해왔으며, '
 '지역마다 특색이 다르다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, '
 '2013년 CNN에서 세계 최고의 요리로 선정되었다.')


In [47]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [48]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QKjiLCIIPDZ2GxWohKEKIzhk', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_vHzOUQs4jQrE7dKcc3byHCAH', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 152, 'total_tokens': 210, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-Bnf69FlxOsDMAK8Sb4aDEjQfHOT7X', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2e3a94d1-cbf3-42f2-b8a4-409cf46af791-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, '

In [49]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])
print(type(tool_message))
print(tool_message)
print("-" * 100)
pprint(tool_message.content)

<class 'langchain_core.messages.tool.ToolMessage'>
content='오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래한 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. 이탈리아 요리는 기원전 4세기부터 발전해왔으며, 지역에 따라 다양한 특색을 지닌다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다.' name='wiki_summary' tool_call_id='call_vHzOUQs4jQrE7dKcc3byHCAH'
----------------------------------------------------------------------------------------------------
('오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래한 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 '
 '박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. 이탈리아 요리는 기원전 4세기부터 발전해왔으며, '
 '지역에 따라 다양한 특색을 지닌다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 '
 '중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다.')


In [50]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_HVyuWpkRpClTu640hy0AyVw7', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bnf6K4zo2CVF2a3BkS4oFnKME4clZ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--b80d7462-3c54-4fd9-9bb1-29930a7bb23a-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_HVyuWpkRpClTu640hy0AyVw7', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 19, 'total_tokens': 139, 'input_token_details': {

### 벡터DB에 저장 후 검색해서 가져오는 Tool
- tool decorator 사용

In [51]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [52]:

from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = [] # [Document, Document]
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크
내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...

메뉴 번호: 2
메뉴 이름: 트러플 리조또
내용:
2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조...


### Ollama Embedding

In [53]:

from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="qwen2.5:1.5b") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("./db/menu_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 4},
)

In [54]:

# 쿼리 테스트
query = "스테이크 메뉴의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 4개
메뉴 번호: 1
메뉴 이름: 시그니처 스테이크

메뉴 번호: 2
메뉴 이름: 트러플 리조또

메뉴 번호: 9
메뉴 이름: 치킨 콘피

메뉴 번호: 3
메뉴 이름: 연어 타르타르



In [55]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("../data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

embeddings_model = OllamaEmbeddings(model="qwen2.5:1.5b")

# FAISS 인덱스 생성
wine_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

wine_db.save_local("./db/wine_db")

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 4},
)

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...


In [56]:

query = "주요 품종이 까베르네 소비뇽 인 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 4개
메뉴 번호: 10
메뉴 이름: 그랜지 2016

메뉴 번호: 4
메뉴 이름: 클로 뒤 발 2016

메뉴 번호: 9
메뉴 이름: 샤토 디켐 2015

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015



### 도구 정의

In [57]:

from langchain_core.tools import tool

# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "./db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_menu_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_menu_func))
print("-"*100)

print("name: ")
print(db_search_menu_func.name)
print("-"*100)

print("description: ")
pprint(db_search_menu_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_menu_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_menu_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant menu '
                'information from the encrypted database.\n'
                'Use this tool only for menu-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_menu_func',
 'type': 'object'}


In [58]:

from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# wine db 벡터 저장소 로드
wine_db = FAISS.load_local(
    "./db/wine_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_wine_func(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_wine_func))
print("-"*100)

print("name: ")
print(db_search_wine_func.name)
print("-"*100)

print("description: ")
pprint(db_search_wine_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_wine_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_wine_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant wine information from the '
 'encrypted database.\n'
 'Use this tool only for wine-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant wine '
                'information from the encrypted database.\n'
                'Use this tool only for wine-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_wine_func',
 'type': 'object'}


In [59]:

# LLM에 도구를 바인딩 (2개의 도구 바인딩)
# 질문을 보고 어떤 DB 내용을 사용할지 LLM이 자동으로 탐색
llm_with_tools = llm.bind_tools(tools=[db_search_menu_func, db_search_wine_func])

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크의 가격과 특징은 무엇인가요? 주요 품종이 카베르네 소비뇽 인 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AQobtmRrCrb8D5bTMYoZV3tm', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'db_search_menu_func'}, 'type': 'function'}, {'id': 'call_RcxXMbiuceYClIaSokiJuasK', 'function': {'arguments': '{"query": "카베르네 소비뇽"}', 'name': 'db_search_wine_func'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 140, 'total_tokens': 198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bnf7ZsuHFcGKzhdj61GoXyvrYcyIJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0a97d7bd-3c5a-4166-a8b1-09c020e448a5-0', tool_calls=[{'name': 'db_search_menu_func', 'args': {'query': '스테이크'}, 'id': '

### Tool 여러개 사용해보기

In [60]:
# @tool : tavily_search_func,db_search_menu_func,db_search_wine_func   
# as_tool()함수 : wiki_summary, 
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
for tool in tools:
    print(tool.name)

tavily_search_func
wiki_summary
db_search_menu_func
db_search_wine_func


In [61]:

from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        pprint(f"{tool_call['name']}: \n{tool_call}")
        print("%"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n") 
    for tool in tool_msgs:
        pprint(tool.name)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

In [62]:

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 해주세요.")

# 응답 출력 
print(response.content)

('db_search_menu_func: \n'
 "{'name': 'db_search_menu_func', 'args': {'query': '시그니처 스테이크'}, 'id': "
 "'call_ldmweoC3afPN4MSuoJA3peW8', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
('db_search_wine_func: \n'
 "{'name': 'db_search_wine_func', 'args': {'query': '시그니처 스테이크'}, 'id': "
 "'call_XXsUxKza8MANA5eJMTw21IUf', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
tool_msgs: 

'db_search_menu_func'
'db_search_wine_func'
----------------------------------------------------------------------------------------------------
### 시그니처 스테이크
- **가격**: 35,000원
- **특징**: 
  - 주 재료: 소고기
  - 조리 방식: 미디엄 레어로 조리된 최상급 한우 등심
  - 식사 경험: 특별히 선정된 주문 소스와 함께 제공되며, 뒷 고기의 깊은 풍미와 함께 다양한 채소와 곁들여져 식사의 품격을 높입니다.

### 추천 와인
1. **그랜지 2016**
   - **가격**: 950,000원
   - **특징**: 
     - 주 재료: 프랑스 론 지역의 최상급 포도
     - 설명: 헌화의 매력적인 느낌을 가진 이 와인은 편안하고 영혼의 깊이를 더해주는 선택입니다. 뚜렷한

In [63]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 파스타의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

('db_search_menu_func: \n'
 "{'name': 'db_search_menu_func', 'args': {'query': '파스타'}, 'id': "
 "'call_NfqK0k27yE8MPVUS2tHAPO9b', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
('wiki_summary: \n'
 "{'name': 'wiki_summary', 'args': {'query': '파스타'}, 'id': "
 "'call_rl9qH8omh6nkWttbecPeexdS', 'type': 'tool_call'}")
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
tool_msgs: 

'db_search_menu_func'
'wiki_summary'
----------------------------------------------------------------------------------------------------
### 파스타 메뉴
다음은 현재 제공되는 파스타 메뉴입니다:

1. **안심 스테이크 살로타**  
   - 가격: 26,000원  
   - 주요 재료: 소고기 안심, 로제 소스, 채리 토마토, 바질 대파
   - 설명: 부드러운 안심 스테이크를 얹게 신선한 로제 소스와 함께 제공됩니다.

2. **시그니처 스파게티**  
   - 가격: 35,000원  
   - 주요 재료: 내장미고기, 로즈마리 간장, 그릴드 아스파라거스
   - 설명: 송로의 풍미가 더욱 진해지는 시그니처 스파게티입니다.

3. **트레첼리 리조또**  
   - 가격: 22,000원  
   - 주요 재료: 이탈리아산 아란치니, 베리반 베이컨, 파르미지

### Few-Shot
- 모델에게 예시를 줌

In [67]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_menu_func", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),    
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. \
                주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_wine_func", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. \
                2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다.\
               주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, \
              주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. \
              특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오,\
               그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the db_search_menu_func tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the db_search_wine_func tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인도 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'db_search_menu_func', 'args': {'query': '스테이크'}, 'id': 'call_N3N7voMFyQ9yAzI1AH3BB2YR', 'type': 'tool_call'}
{'name': 'db_search_wine_func', 'args': {'query': '스테이크'}, 'id': 'call_AzLRSku9dE9Yj7AhYGL4z9n4', 'type': 'tool_call'}


In [68]:

# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_pJy7nrViqb8P5eMNyJtj2U1m', 'type': 'tool_call'}
{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_zXz2JqjYzGfnomMi1zYLn3um', 'type': 'tool_call'}


In [69]:

from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the db_search_menu_func tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the db_search_wine_func tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

In [70]:

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        # [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해 주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

db_search_menu_func: 
{'name': 'db_search_menu_func', 'args': {'query': '스테이크'}, 'id': 'call_mRpRPS8PtarBqBk7zEGnZ9xI', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
db_search_wine_func: 
{'name': 'db_search_wine_func', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_ghRJfq3w1umAPbRyrPtkGApJ', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(id='79406506-abd4-4973-a1c3-bb6f0224a57c', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 8, 'menu_name': '안심 스테이크 샐러드'}, page_content='8. 안심 스테이크 샐러드\\n   • 가격: ₩26,000\\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.'), Document(id='80d7be45-674c-4a54-ad87-7ca2097aa5e9', metadata={'source': '../data/restaurant_menu

In [71]:

# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 실제로 존재하는 서울 강남의 파스타 맛집을 정확하게 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_JKmqO40j4jdgQyg4iYsJQKYC', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tavily_search_func: 
{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_0cP0UcKeYKHpds0zFbH8c6GO', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content='오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래한 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. 이탈리아 요리는 기원전 4세기부터 발전해왔으며, 지역마다 특색이 다르다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다.', name='wiki_summary', tool_call_id='call_JKmqO40j4jdgQyg4iYsJQKYC'), ToolMessage(content='<Document href="https://zdnet.co.kr/view/?no=20180302141956"/>\n△위치: 서울 강남구 선릉로 52길 33 영업시간: 매일 15:0 - 02:00 △가격: 대파 파스타 2만2천원, 쥬키니 후라이 

### LangChain Agent

In [72]:

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the db_search_menu_func tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the db_search_wine_func tool.
        4. If additional web searches are needed or for the most up-to-date information, use the tavily_search_func tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [73]:

# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]

#llm = ChatOpenAI(model="gpt-4o")
# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     temperature=0.3,
#     api_key=
# )

agent = create_tool_calling_agent(llm, tools, agent_prompt)
print(type(agent))

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
print(type(agent_executor))

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain.agents.agent.AgentExecutor'>


In [74]:

# AgentExecutor 실행

#query = "안심 스테이크의 가격과 특징은 무엇인가요? 그리고 와인도 추천 해주세요."
query = "스테이크의 가격과 특징은 무엇인가요? 주요 품종이 카베르네 소비뇽 인 와인을 추천해주세요."
agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `db_search_menu_func` with `{'query': '스테이크'}`


[Document(id='79406506-abd4-4973-a1c3-bb6f0224a57c', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 8, 'menu_name': '안심 스테이크 샐러드'}, page_content='8. 안심 스테이크 샐러드\n   • 가격: ₩26,000\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.'), Document(id='80d7be45-674c-4a54-ad87-7ca2097aa5e9', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 1, 'menu_name': '시그니처 스테이크'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(id='20cb1ca0-64f8-464b-87e0-04c20c45d156', metadata={'source': '../data/restaurant_menu.txt', 'menu_number': 7, 'menu_name': '랍

### Gradio 활용

In [75]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-2:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# 데모 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [76]:

# 데모 종료
demo.close()

Closing server running on port: 7860
